In [45]:
import pandas as pd

# Load the dataset
df = pd.read_csv("LoanDataTape.csv")



In [46]:
df

,loan number,loan amount,interest_rate,start_date,term,payment frequency,CPR (Conditional Prepayment Rate)
0,1,"35,000.00",8%,9/1/2023,36,Monthly,5%
1,2,"40,000.00",8%,10/1/2023,12,Monthly,5%
2,3,"40,000.00",8%,11/1/2023,48,Monthly,5%
3,4,"40,000.00",8%,12/1/2023,60,Monthly,5%
4,5,"40,000.00",8%,1/1/2024,72,Monthly,5%
5,6,"40,000.00",8%,2/1/2024,60,Monthly,5%
6,7,"40,000.00",8%,3/1/2024,72,Monthly,5%
7,8,"40,000.00",8%,4/1/2024,60,Monthly,5%
8,9,"40,000.00",8%,5/1/2024,72,Monthly,5%
9,10,"40,000.00",8%,6/1/2024,60,Monthly,5%


In [47]:
df.columns

Index(['loan number', 'loan amount', 'interest_rate', 'start_date', 'term ',
       'payment frequency', 'CPR (Conditional Prepayment Rate)'],
      dtype='object')

In [48]:
df.columns = ['loan_number', 'loan_amount', 'interest_rate', 'start_date', 'term ',
       'payment_frequency', 'CPR']

In [49]:
df.columns

Index(['loan_number', 'loan_amount', 'interest_rate', 'start_date', 'term ',
       'payment_frequency', 'CPR'],
      dtype='object')

In [41]:
import numpy as np

def calculate_amortization_schedule(principal, rate, term, payment_frequency='Monthly', cpr=0.05, start_date='2023-01-01'):
    # Adjust interest rate and periods based on frequency
    if payment_frequency == 'Monthly':
        periods = term
        rate /= 12
        cpr_rate = cpr / 12
    else:
        raise ValueError("Only 'Monthly' payment frequency is supported in this implementation.")

    # Calculate fixed payment amount (assuming a fixed-rate loan)
    if rate > 0:
        payment = principal * (rate * (1 + rate) ** periods) / ((1 + rate) ** periods - 1)
    else:
        payment = principal / periods

    # Initialize amortization schedule
    schedule = []
    balance = principal
    current_date = pd.to_datetime(start_date)

    for period in range(1, int(periods + 1)):
        # Apply CPR to reduce principal
        prepayment_amount = balance * cpr_rate
        balance -= prepayment_amount

        # Calculate interest and principal payments
        interest_payment = balance * rate
        principal_payment = payment - interest_payment
        balance -= principal_payment

        # Avoid negative balance due to rounding errors
        balance = max(balance, 0)

        # Append period details
        schedule.append({
            "Date": current_date,
            "Loan ID": period,  # replace with actual loan ID in loop
            "Opening Balance": round(balance + principal_payment + prepayment_amount, 2),
            "Principal Payment": round(principal_payment, 2),
            "Interest Payment": round(interest_payment, 2),
            "Prepayment Amount": round(prepayment_amount, 2),
            "Closing Balance": round(balance, 2)
        })

        # Stop if balance is zero
        if balance == 0:
            break

        # Update date to next period
        current_date += pd.DateOffset(months=1)

    return schedule


In [50]:
type('interest_rate')

str

In [51]:
df.columns

Index(['loan_number', 'loan_amount', 'interest_rate', 'start_date', 'term ',
       'payment_frequency', 'CPR'],
      dtype='object')

In [52]:
# Remove the '%' sign and convert the interest rate to a float
df['interest_rate'] = df['interest_rate'].str.replace('%', '').astype(float) / 100


In [53]:
df.columns = ['loan_number', 'loan_amount', 'interest_rate', 'start_date', 'term',
       'payment_frequency', 'CPR']


In [54]:
df['CPR'] = df['CPR'].str.replace('%', '').astype(float) / 100

In [55]:
# Remove any extraneous text or symbols, then convert to float
df['term'] = df['term'].replace('[^0-9]', '', regex=True).astype(float)


In [56]:
df['loan_amount'] = df['loan_amount'].replace('[\$,]', '', regex=True).astype(float)

<>:1: SyntaxWarning: invalid escape sequence '\$'
<>:1: SyntaxWarning: invalid escape sequence '\$'
C:\Users\User\AppData\Local\Temp\ipykernel_20992\3295780262.py:1: SyntaxWarning: invalid escape sequence '\$'
  df['loan_amount'] = df['loan_amount'].replace('[\$,]', '', regex=True).astype(float)


In [31]:
# Convert term from years to months if needed
#df['term'] = df['term'] /12

In [57]:
#Convert 'start'_date' to date, 
df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce', format='%m-%d-%Y')

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   loan_number        10 non-null     int64         
 1   loan_amount        10 non-null     float64       
 2   interest_rate      10 non-null     float64       
 3   start_date         0 non-null      datetime64[ns]
 4   term               10 non-null     float64       
 5   payment_frequency  10 non-null     object        
 6   CPR                10 non-null     float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 692.0+ bytes


In [59]:
all_schedules = []

for index, row in df.iterrows():
    # Calculate the amortization schedule for each loan
    schedule = calculate_amortization_schedule(
        principal=row['loan_amount'],
        rate=row['interest_rate'] / 100,  # Convert percentage to decimal
        term=row['term'],
        payment_frequency=row['payment_frequency'],
        cpr=row['CPR'] / 100,
        start_date=row['start_date']
    )
    
    # Add loan number to each entry in schedule
    for entry in schedule:
        entry["Loan ID"] = row['loan_number']
    
    # Append to all schedules
    all_schedules.extend(schedule)


In [60]:
# Convert list of schedules to DataFrame
schedule_df = pd.DataFrame(all_schedules)

# Group by date and aggregate amounts
consolidated_schedule = schedule_df.groupby("Date").agg({
    "Opening Balance": "sum",
    "Principal Payment": "sum",
    "Interest Payment": "sum",
    "Prepayment Amount": "sum",
    "Closing Balance": "sum"
}).reset_index()


In [62]:
consolidated_schedule.to_csv("Consolidated_Amortization_Schedule.csv", index=False)

In [75]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv("LoanDataTape.csv")

df.columns = ['loan number', 'loan amount', 'interest_rate', 'start_date', 'term','payment frequency', 'CPR (Conditional Prepayment Rate)']
# Clean up 'interest_rate' column: remove '%' and convert to float
df['interest_rate'] = df['interest_rate'].str.replace('%', '').astype(float) / 100
#CPR STRING TO FLOAT type convertion 

df['CPR (Conditional Prepayment Rate)'] = df['CPR (Conditional Prepayment Rate)'].str.replace('%', '').astype(float) / 100
df['loan amount'] = df['loan amount'].replace('[\$,]', '', regex=True).astype(float)
# Clean and convert loan amount



def calculate_amortization_schedule(principal, rate, term, payment_frequency='Monthly', cpr=0.05, start_date='2023-01-01'):
    # Adjust interest rate and periods based on frequency
    if payment_frequency == 'Monthly':
        periods = term
        rate /= 12
        cpr_rate = cpr / 12
    else:
        raise ValueError("Only 'Monthly' payment frequency is supported in this implementation.")

    # Calculate fixed payment amount (assuming a fixed-rate loan)
    if rate > 0:
        payment = principal * (rate * (1 + rate) ** periods) / ((1 + rate) ** periods - 1)
    else:
        payment = principal / periods

    # Initialize amortization schedule
    schedule = []
    balance = principal
    current_date = pd.to_datetime(start_date)

    for period in range(1, periods + 1):
        # Apply CPR to reduce principal
        prepayment_amount = balance * cpr_rate
        balance -= prepayment_amount

        # Calculate interest and principal payments
        interest_payment = balance * rate
        principal_payment = payment - interest_payment
        balance -= principal_payment

        # Avoid negative balance due to rounding errors
        balance = max(balance, 0)

        # Append period details
        schedule.append({
            "Date": current_date,
            "Loan ID": period,  # replace with actual loan ID in loop
            "Opening Balance": round(balance + principal_payment + prepayment_amount, 2),
            "Principal Payment": round(principal_payment, 2),
            "Interest Payment": round(interest_payment, 2),
            "Prepayment Amount": round(prepayment_amount, 2),
            "Closing Balance": round(balance, 2)
        })

        # Stop if balance is zero
        if balance == 0:
            break

        # Update date to next period
        current_date += pd.DateOffset(months=1)

    return schedule

# Generate amortization schedule for each loan
all_schedules = []

for index, row in df.iterrows():
    # Clean and convert loan amount
    

    # Calculate the amortization schedule for each loan
    # Ensure rate is correctly formatted as a float before passing it
    schedule = calculate_amortization_schedule(
        principal=float(row['loan amount']),
        rate=float(row['interest_rate']) / 100,  # Convert percentage to decimal
        term=int(row['term']),  # Make sure the term is an integer
        payment_frequency=row['payment frequency'],
        cpr=float(row['CPR (Conditional Prepayment Rate)']) / 100,  # Convert CPR to decimal
        start_date=row['start_date']

    )
    
    # Add loan number to each entry in schedule
    for entry in schedule:
        entry["Loan ID"] = row['loan number']
    
    # Append to all schedules
    all_schedules.extend(schedule)

# Convert list of schedules to DataFrame
schedule_df = pd.DataFrame(all_schedules)

# Consolidate schedules by date
consolidated_schedule = schedule_df.groupby("Date").agg({
    "Opening Balance": "sum",
    "Principal Payment": "sum",
    "Interest Payment": "sum",
    "Prepayment Amount": "sum",
    "Closing Balance": "sum"
}).reset_index()

# Save the consolidated schedule to CSV
consolidated_schedule.to_csv("Consolidated_Amortization_Schedule.csv", index=False)


<>:13: SyntaxWarning: invalid escape sequence '\$'
<>:13: SyntaxWarning: invalid escape sequence '\$'
C:\Users\User\AppData\Local\Temp\ipykernel_20992\4253514908.py:13: SyntaxWarning: invalid escape sequence '\$'
  df['loan amount'] = df['loan amount'].replace('[\$,]', '', regex=True).astype(float)


In [76]:
pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/317.3 MB ? eta -:--:--
     --------------------------------------- 3.9/317.3 MB 19.5 MB/s eta 0:00:17
     - ------------------------------------- 9.4/317.3 MB 24.5 MB/s eta 0:00:13
     - ------------------------------------ 12.8/317.3 MB 21.8 MB/s eta 0:00:14
     -- ----------------------------------- 18.9/317.3 MB 23.4 MB/s eta 0:00:13
     -- ----------------------------------- 24.1/317.3 MB 23.9 MB/s eta 0:00:13
     --- ---------------------------------- 27.3/317.3 MB 22.4 MB/s eta 0:00:13
     --- ---------------------------------- 29.6/317.3 MB 20.9 MB/s eta 0:00:14
     --- ---------------------------------- 32.0/317.3 MB 19.5 MB/s eta 0:00:15
     ---- --------------------------------- 34.6/317.3 MB 18.8 MB/s eta 0:00:16
     ---- --------------------------------- 37.2/317.3 MB 18.3 MB/s eta 0:00:16
     ---- --------------------------------- 39.8/


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, lit
import pandas as pd

# Initialize Spark session
spark = SparkSession.builder.appName("LoanAmortization").getOrCreate()

# Step 1: Read Data
df = spark.read.csv("path_to_your_csv_file.csv", header=True, inferSchema=True)

# Clean and convert relevant columns
df = df.withColumn("loan amount", col("loan amount").cast("string"))
df = df.withColumn("loan amount", col("loan amount").replace(",", "").cast("float"))
df = df.withColumn("interest_rate", col("interest_rate").cast("float") / 100)
df = df.withColumn("term", col("term").cast("int"))
df = df.withColumn("CPR (Conditional Prepayment Rate)", col("CPR (Conditional Prepayment Rate)").cast("float") / 100)

# Step 2: Define Amortization Schedule Calculation as a Pandas UDF
def calculate_amortization_schedule(row):
    principal = row['loan amount']
    rate = row['interest_rate']
    term = row['term']
    payment_frequency = row['payment frequency']
    cpr = row['CPR (Conditional Prepayment Rate)']
    start_date = row['start_date']

    # Calculate the amortization schedule as before (using the logic you implemented)
    schedule = []  # Fill with amortization logic output for each period

    # Return as a Pandas DataFrame for each row in Spark
    return pd.DataFrame(schedule)

# Step 3: Apply the UDF and Aggregate
# Use explode to flatten the amortization schedule
amortized_df = df.mapInPandas(calculate_amortization_schedule, schema="...")  # Define schema based on your output

# Group by date and aggregate payments, etc.
aggregated_df = amortized_df.groupBy("date").agg({"payment": "sum", "principal": "sum", "interest": "sum"})

# Step 4: Write Output to CSV
aggregated_df.write.csv("aggregated_amortization_schedule.csv", header=True)

# Stop Spark session
spark.stop()
